In [25]:
# from __future__ import division
from __future__ import print_function
%matplotlib inline

# ignore deprecation warnings in sklearn
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import sys

src_dir=os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(src_dir)

from data.multilabel import multilabel_sample_dataframe, multilabel_train_test_split
from features.SparseInteractions import SparseInteractions
from models.metrics import multi_multi_log_loss

In [2]:
np.__version__

'1.16.4'

#load data (categorical variable, numeric value, text format)

In [3]:
path_to_training_data = os.path.join(os.pardir, 'data', 'TrainingData.csv')
path_to_test_data = os.path.join(os.pardir,'data','TestData.csv')

In [4]:
df = pd.read_csv(path_to_training_data, index_col=0)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 400277 entries, 134338 to 415831
Data columns (total 25 columns):
Function                  400277 non-null object
Use                       400277 non-null object
Sharing                   400277 non-null object
Reporting                 400277 non-null object
Student_Type              400277 non-null object
Position_Type             400277 non-null object
Object_Type               400277 non-null object
Pre_K                     400277 non-null object
Operating_Status          400277 non-null object
Object_Description        375493 non-null object
Text_2                    88217 non-null object
SubFund_Description       306855 non-null object
Job_Title_Description     292743 non-null object
Text_3                    109152 non-null object
Text_4                    53746 non-null object
Sub_Object_Description    91603 non-null object
Location_Description      162054 non-null object
FTE                       126071 non-null float64
Func

For our purpose and computational efficiency, we will sample down to 10,000 rows from 400,277rows so that it's easy to quick to run our analysis. We will also create dummy variables for our labels and split our sampel dataset into a training set and a test set.

In [10]:
LABELS = ['Function',
          'Use',
          'Sharing',
          'Reporting',
          'Student_Type',
          'Position_Type',
          'Object_Type', 
          'Pre_K',
          'Operating_Status']
NON_LABELS = [c for c in df.columns if c not in LABELS]
SAMPLE_SIZE = 10000
sampling = multilabel_sample_dataframe(df, 
                            pd.get_dummies(df[LABELS]),
                            size = SAMPLE_SIZE,
                            min_count=25,
                            seed=43)

dummy_labels = pd.get_dummies(sampling[LABELS])
X_train, X_test, y_train, y_test = multilabel_train_test_split(sampling[NON_LABELS],
                                                               dummy_labels,
                                                               0.2,
                                                               min_count=3,
                                                               seed=43)


#create processing tools for text and numeric data. The combine_text_columns function will take a DataFrame of text columns and return a single series where all of the text in the columns has been joined together.
FunctionTransformer objects that select our text and numeric data from the dataframe.
scoring method that uses the multi_multi_log_loss function that is the evaluation metric for the competition.

In [11]:
NUMERIC_COLUMNS = ['FTE', 'Total']

def combine_text_columns(data_frame, to_drop = NUMERIC_COLUMNS + LABELS):
    """ Takes the dataset as read in, drops the non-feature, non-text columns and
        then combines all of the text columns into a single vector that has all of
        the text for a row.
        
        :param data_frame: The data as read in with read_csv (no preprocessing necessary)
        :param to_drop (optional): Removes the numeric and label columns by default.
    """
    # drop non-text columns that are in the df
    to_drop = set(to_drop) & set(data_frame.columns.tolist())
    text_data = data_frame.drop(to_drop, axis=1)
    
    # replace NaN with Blanks
    text_data.fillna("", inplace=True)
    
    # joins all of the text items in a row (axis=1) with a space in between
    return text_data.apply(lambda x: " ".join(x), axis =1)

In [12]:
combine_text_columns(df, to_drop=NUMERIC_COLUMNS + LABELS)

134338       Teacher-Elementary        KINDERGARTEN  KIN...
206341    CONTRACTOR SERVICES BOND EXPENDITURES BUILDING...
326408    Personal Services - Teachers   TCHER 2ND GRADE...
364634    EMPLOYEE BENEFITS TEACHER SUBS GENERAL FUND Te...
47683     TEACHER COVERAGE FOR TEACHER TEACHER SUBS GENE...
229958    CONTRA BENEFITS  GENERAL FUND Custodian - PT -...
417668    EDUCATIONAL SPECIAL EDUCATION INSTRUCTION LOCA...
126378    EMPLOYEE BENEFITS  GENERAL FUND Sub Manager, F...
275539    EMPLOYEE BENEFITS  GENERAL FUND Teacher, Eleme...
85262     EMPLOYEE BENEFITS TEACHER SUBS GENERAL FUND Te...
304569    EQUIPMENT *  Support Services - Administration...
330504    SUPPLIES  PRIMARY GRADES PROGRAM       INSTRUC...
84272     Personal Services - Teachers   TCHER P E (ELEM...
64760     Regular *  Special Instruction    Certificated...
21870     OTHER PERSONAL SERVICES          SUB TEACHER A...
18698     Salaries And Wages For Substitute Professional...
169454    SUPPLIES                      

In [13]:
from sklearn.preprocessing import FunctionTransformer

get_text_data = FunctionTransformer(combine_text_columns, validate=False)
get_numeric_data = FunctionTransformer(lambda x: x[NUMERIC_COLUMNS], validate=False)

In [14]:
get_text_data.fit_transform(sampling.head(5))
get_numeric_data.fit_transform(sampling.head(5))

,FTE,Total
38,NaN,653.460000
70,NaN,2153.530000
198,NaN,-8291.860000
209,NaN,618.290000
614,0.71,21747.666875


In [30]:
from sklearn.metrics.scorer import make_scorer
log_loss_scorer = make_scorer(multi_multi_log_loss)

# TRAIN MODEL PIPELINE
Now we'll train the final pipeline from the course that takes text and numeric data, does the necessary preprocessing, and trains the classifier.

In [31]:
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import Imputer
from sklearn.feature_extraction.text import HashingVectorizer

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import MaxAbsScaler



In [32]:


# create the pipeline object
pl = Pipeline([
        ('union', FeatureUnion(
            transformer_list = [
                ('numeric_features', Pipeline([
                    ('selector', get_numeric_data),
                    ('imputer', Imputer())
                ])),
                ('text_features', Pipeline([
                    ('selector', get_text_data),
                    ('vectorizer', HashingVectorizer(alternate_sign=False, norm=None, binary=False,ngram_range=(1, 2))),
                    ('dim_red', SelectKBest(chi2, chi_k))
                ]))
             ]
        )),
        ('int', SparseInteractions(degree=2)),
        ('scale', MaxAbsScaler()),
        ('clf', OneVsRestClassifier(LogisticRegression()))
    ])

# fit the pipeline to our training data
pl.fit(X_train, y_train.values)

# print the score of our trained pipeline on our test set
print("Logloss score of trained pipeline: ", log_loss_scorer(pl, X_test, y_test.values))
               

Logloss score of trained pipeline:  2.1864848617308343
CPU times: user 16min 39s, sys: 8.58 s, total: 16min 47s
Wall time: 4min 40s


In [19]:
import sklearn
print(sklearn.__version__)

0.21.2


In [24]:
path_to_holdout_data=os.path.join (os.pardir, 'data', 'TestData.csv')

# Load holdout data
#holdout = pd.read_csv(path_to_holdout_data, index_col=0)
holdout = pd.read_csv(path_to_holdout_data, index_col=0)
# Make predictions
predictions = pl.predict_proba(holdout)
#predictions = pl.predict_proba(holdout)

# Format correctly in new DataFrame: prediction_df
prediction_df = pd.DataFrame(columns = pd.get_dummies(df[LABELS]).columns,
                            index=holdout.index,
                            data=predictions)

#prediction_df = pd.DataFrame(columns=pd.get_dummies(df[LABELS]).columns,
#                             index=holdout.index,
#                             data=predictions)


# Save prediction_df to csv called "predictions.csv"
prediction_df.to_csv("predictions.csv")
#prediction_df.to_csv("predictions.csv")